In [1]:
from sogaPreprocessor import *
from producecfg import *
from smoothcfg import *
from libSOGA import *

In [2]:
compiledFile=compile2SOGA_text('x = gm([0.1, 0.1, 0.8], [1., 1., 2.], [1., 1., 1.]); ')
cfg = produce_cfg_text(compiledFile)
smooth_cfg(cfg)

params_dict = {}

output_dist = start_SOGA(cfg, params_dict)
output_dist

Dist<['x'],pi: tensor([[0.1000],
        [0.1000],
        [0.8000]])
mu: tensor([[1.],
        [1.],
        [2.]])
sigma: tensor([[[1.]],

        [[1.]],

        [[1.]]])>

In [3]:
compiledFile=compile2SOGA_text('x = gm([0.1, 0.1, 0.8], [1., 1., 2.], [1., 1., 1.]); prune(2);')
cfg = produce_cfg_text(compiledFile)
smooth_cfg(cfg)

params_dict = {}

output_dist = start_SOGA(cfg, params_dict)
output_dist

Pruning Dist<['x'],pi: tensor([[0.1000],
        [0.1000],
        [0.8000]])
mu: tensor([[1.],
        [1.],
        [2.]])
sigma: tensor([[[1.]],

        [[1.]],

        [[1.]]])>  with  classic  Kmax= 2


Dist<['x'],pi: tensor([[0.8000],
        [0.2000]])
mu: tensor([[2.],
        [1.]])
sigma: tensor([[[1.]],

        [[1.]]])>

In [4]:
dist = Dist(['x', 'y'], GaussianMix(torch.tensor([[0.1], [0.1], [0.8]]), torch.tensor([[-0.1, 2.], [0.1, 2.], [1., 1.]]), torch.stack([torch.eye(2), torch.eye(2), torch.eye(2)])))

classic_prune(dist, 2)

Dist<['x', 'y'],pi: tensor([[0.8000],
        [0.2000]])
mu: tensor([[1., 1.],
        [0., 2.]])
sigma: tensor([[[1.0000, 0.0000],
         [0.0000, 1.0000]],

        [[1.0100, 0.0000],
         [0.0000, 1.0000]]])>